In [1]:
import argparse
import os
import numpy as np
import math
import itertools
import time
import datetime
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

In [41]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [42]:
transforms_A = [ transforms.Resize((256, 256), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
transforms_B = [ transforms.Resize((256, 256), Image.BICUBIC),
                transforms.ToTensor() ]
dataloader = DataLoader(ImageDataset("/media/arg_ws3/5E703E3A703E18EB/data/subt_real/", transforms_A=transforms_A, transforms_B=transforms_B),
                        batch_size=1, shuffle=True, num_workers=8)
data_iter = iter(dataloader)

In [47]:
generator = GeneratorUNet()
discriminator = Discriminator()
if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()

In [77]:
batch = next(data_iter)
real_A = Variable(batch['B'].type(Tensor))
real_B = Variable(batch['A'].type(Tensor))

In [78]:
patch = (1, 256//2**4, 256//2**4)
patch

(1, 16, 16)

In [79]:
valid = Variable(Tensor(np.ones((real_A.size(0), *patch))), requires_grad=False)
fake = Variable(Tensor(np.zeros((real_A.size(0), *patch))), requires_grad=False)

In [80]:
fake_B = generator(real_A)
fake_B

tensor([[[[-0.0121, -0.0574, -0.0312,  ..., -0.1188, -0.0714,  0.0135],
          [ 0.0452, -0.1303, -0.2079,  ..., -0.0747, -0.1799, -0.0830],
          [ 0.0427,  0.0930, -0.1316,  ...,  0.1492,  0.0406, -0.0753],
          ...,
          [ 0.1257, -0.0198, -0.1558,  ..., -0.1002,  0.0152, -0.0852],
          [ 0.0724,  0.2313,  0.2237,  ...,  0.1921,  0.1879, -0.0705],
          [ 0.1089,  0.0540,  0.1451,  ...,  0.2091,  0.1530, -0.0341]],

         [[-0.0611, -0.0362, -0.0279,  ...,  0.1552,  0.0966, -0.0161],
          [ 0.0166,  0.0379,  0.1096,  ...,  0.0389,  0.0352, -0.0641],
          [-0.0095,  0.2534,  0.3360,  ...,  0.2434,  0.1572, -0.1094],
          ...,
          [-0.1773, -0.1123,  0.0505,  ...,  0.1430, -0.0748, -0.2578],
          [-0.0132,  0.1588,  0.0290,  ...,  0.1171,  0.0944, -0.0645],
          [ 0.0121,  0.0570,  0.0333,  ...,  0.0544,  0.1803,  0.0053]],

         [[ 0.0123,  0.0340,  0.1028,  ..., -0.0748,  0.0720,  0.0566],
          [-0.1092, -0.1949, -